<a href="https://colab.research.google.com/github/reban87/ML-Projects/blob/main/Task2_MCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 2: Multiclass Text Classification | RPA Labs | Rebanta Aryal | 1st April 2022

[```ktrain```](https://amaiya.github.io/ktrain/index.html) is wrapper library for tensorflow and keras and can be used to implement in HuggingFace. 

In [ ]:
!pip install ktrain

     |████████████████████████████████| 25.3 MB 58.2 MB/s 
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 981 kB 51.8 MB/s 
     |████████████████████████████████| 263 kB 49.6 MB/s 
     |████████████████████████████████| 2.8 MB 29.9 MB/s 
     |████████████████████████████████| 1.2 MB 38.0 MB/s 
     |████████████████████████████████| 468 kB 44.3 MB/s 
     |████████████████████████████████| 596 kB 46.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.3 MB/s 
     |████████████████████████████████| 895 kB 37.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
  Created wheel for ktrain: filename=ktrain-0.30.0-py3-none-any.whl size=25309161 sha256=ba117c239815aefc6fa74cc7c0cc740c34e4853d9392ffc07d5154d42a020e44
  Stored in directory: /root/.cache/pip/wheels/25/00/ae/80d1b19dd02bc2e6bc9bac32ba7dbaa59aad789e93274e4868
  Created wheel for keras-bert: filename=keras_bert-0.89.0-py3-none-any.whl size=33517 sha256=7a249867

To use a particular set of GPU devices, the CUDA_VISIBLE_DEVICES environment variable has been used.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

Let us use [20news group](https://www.kaggle.com/datasets/crawford/20-newsgroups) datasets from Sklearn.

In [ ]:
import ktrain
from ktrain import text
from sklearn.datasets import fetch_20newsgroups

since 20 NEWS groups has 20 classes. Lets take only few of them, Since training all the dataset will be computationally expensive

In [ ]:
categories=['comp.graphics','rec.motorcycles','sci.med','rec.sport.baseball','sci.electronics']

Training and testing datasets have been generated using the method ```fetch_20newsgroups()```


In [ ]:
train=fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=0
)

In [ ]:
test=fetch_20newsgroups(
    subset='test',
    categories=categories,
    shuffle=True,
    random_state=0
)

Since our main concern is to classify in multiple classes, in our case we have 5 different classes.

In [ ]:
test.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [ ]:
test.target

array([4, 3, 3, ..., 1, 1, 2])

In [ ]:
test.target_names

['comp.graphics',
 'rec.motorcycles',
 'rec.sport.baseball',
 'sci.electronics',
 'sci.med']

In [ ]:
X_train=train.data
y_train=train.target

X_test=test.data
y_test=test.target

In [ ]:
len(X_train)

2964

In [ ]:
len(X_test)

1973

Build the model using ```Distilbert-base-uncased``` model as it is small and training is faster as well

In [ ]:
model_name='distilbert-base-uncased'
trans=text.Transformer(model_name,maxlen=512,class_names=categories)

In [ ]:
train_data=trans.preprocess_train(X_train,y_train)
test_data=trans.preprocess_test(X_test,y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 233
	95percentile : 559
	99percentile : 1150


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 251
	95percentile : 618
	99percentile : 1425


In [ ]:
model=trans.get_classifier()

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [ ]:
learner=ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=16)

It is always difficult to find out the best learning rate for our model. Exploring different articles regarding the text classification, setting the value of learning rate ```0.0001``` gives fair result also training for more than 2 epochs takes more time.

In [ ]:
learner.fit_onecycle(1e-4,1)



begin training using onecycle policy with max lr of 0.0001...
186/186 [==============================] - 419s 2s/step - loss: 0.5031 - accuracy: 0.8590 - val_loss: 0.2376 - val_accuracy: 0.9290


Training for a single epoch as well, the model has the accuracy of 85.95% with validation accuracy of 92.90%. Looking at confusion matrix, precision, recall and f1 score, it looks fine.

In [ ]:
learner.validate(class_names=categories)

                    precision    recall  f1-score   support

     comp.graphics       0.94      0.89      0.91       389
   rec.motorcycles       0.92      0.94      0.93       398
           sci.med       0.99      0.95      0.97       397
rec.sport.baseball       0.88      0.90      0.89       393
   sci.electronics       0.92      0.96      0.94       396

          accuracy                           0.93      1973
         macro avg       0.93      0.93      0.93      1973
      weighted avg       0.93      0.93      0.93      1973



array([[347,   5,   1,  26,  10],
       [  2, 374,   1,  12,   9],
       [  4,   7, 379,   0,   7],
       [ 13,  21,   0, 352,   7],
       [  5,   1,   0,   9, 381]])

In [ ]:
predict_data=ktrain.get_predictor(learner.model,preproc=trans)

In [ ]:
x='I have a friend name Rebanta and he has a fever'

In [ ]:
predict_data.predict(x)

'sci.electronics'

In [ ]:
y='digital multimeter can be used to find the voltage and current'

In [ ]:
predict_data.predict(y)

'rec.sport.baseball'

In [ ]:
z="I have a 42 years old friend, misdiagnosed as having osteopporosos for two years"

In [ ]:
predict_data.predict(z)

'sci.electronics'

In [ ]:
m="Bike riding in very high traffic highway is dangerous"

In [ ]:
predict_data.predict(m)

'rec.motorcycles'